### <font color = brown> Import Libraries and Join the source </font>

In [1]:
# Import Libraries
import random
import re
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
# Importing test_train_split from sklearn library
from sklearn.model_selection import train_test_split
###teradata_connect notes can be found on readme
import teradata_connect

In [2]:
# Read data from Teradata
query = """
select A.CNCRN_ID
    ,A.RECVD_D
    ,A.CNCRN_DS
    from (
    select CNCRN_ID, MAX(START_TS) AS MAX_START_TS
    from PRD_RACOE_PI_CNS_VR.VR_CFMS_CNCRN  
    GROUP BY CNCRN_ID
    where  CAST (RECVD_D AS DATE)  between '2019-07-01' and '2019-12-31') as B
    inner join PRD_RACOE_PI_CNS_VR.VR_CFMS_CNCRN  as A on B.CNCRN_ID = A.CNCRN_ID and A.START_TS = B.MAX_START_TS
"""
teradata_df2 = teradata_connect.read_teradata_query(query)
print(teradata_df2.shape)

['SQLite', 'SQLite3', 'PostgreSQL ANSI', 'PostgreSQL Unicode']
Using JDBC connect to the Teradata


java.sql.SQLExceptionPyRaisable: java.sql.SQLException: [Teradata Database] [TeraJDBC 16.20.00.13] [Error 8017] [SQLState 28000] The UserId, Password or Account is invalid.

In [4]:
teradata_df2.head()

CNCRN_ID     RECVD_D                                           CNCRN_DS
0  COM-908270  2019-07-19  Case Management- Lending customer requests for...
1  COM-937313  2019-12-11  You are dissatisfied with delay in assessment ...
2  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...
3  COM-936072  2019-12-05  Customer, would like to cancel his Bpay disput...
4  COM-922281  2019-09-26  I believe in protecting and improving our natu...

In [5]:
# Read excel data
exl_data_df1 = pd.read_excel("Code_Breaches_JuneDec2019.xlsx")
exl_data_df1['CNCRN_ID'] = exl_data_df1['CNCRN_ID'].str.strip()
exl_data_df1.head()

BCOP_Breach1                    BCOP_Breach2  \
0         43 When we are recovering a debt                             NaN   
1        26 What we will tell and give you  04 Trained and competent staff   
2            05 Protecting confidentiality                             NaN   
3         43 When we are recovering a debt                             NaN   
4  36 Closing any of your banking services                             NaN   

  BCOP_Breach3 BCOP_Breach4    CNCRN_ID  Breaches  
0          NaN          NaN  COM-904302         1  
1          NaN          NaN  COM-904365         1  
2          NaN          NaN  COM-904432         1  
3          NaN          NaN  COM-904483         1  
4          NaN          NaN  COM-904542         1

In [6]:
print(exl_data_df1.shape)
print(len(exl_data_df1["CNCRN_ID"].unique()))

(302, 6)
302


In [7]:
# Inner join on Concern ID
master_df = pd.merge(teradata_df2, exl_data_df1, on='CNCRN_ID', how='outer', indicator= True)
master_df.head()

CNCRN_ID     RECVD_D                                           CNCRN_DS  \
0  COM-908270  2019-07-19  Case Management- Lending customer requests for...   
1  COM-937313  2019-12-11  You are dissatisfied with delay in assessment ...   
2  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...   
3  COM-936072  2019-12-05  Customer, would like to cancel his Bpay disput...   
4  COM-922281  2019-09-26  I believe in protecting and improving our natu...   

                          BCOP_Breach1 BCOP_Breach2 BCOP_Breach3 BCOP_Breach4  \
0                                  NaN          NaN          NaN          NaN   
1                                  NaN          NaN          NaN          NaN   
2  09 Communication between us and you          NaN          NaN          NaN   
3                                  NaN          NaN          NaN          NaN   
4                                  NaN          NaN          NaN          NaN   

   Breaches     _merge  
0       NaN  left_only  
1       NaN  left_only  
2       1.0       both  
3       NaN  left_only  
4       NaN  left_only

In [8]:
# Evaluate all columns in the merged dataframe
master_df.columns

Index(['CNCRN_ID', 'RECVD_D', 'CNCRN_DS', 'BCOP_Breach1', 'BCOP_Breach2',
       'BCOP_Breach3', 'BCOP_Breach4', 'Breaches', '_merge'],
      dtype='object')

In [9]:
# Dimension of merged dataframe
master_df.shape

(35664, 9)

In [10]:
master_df["_merge"].value_counts()

left_only     35362
both            302
right_only        0
Name: _merge, dtype: int64

### <font color = brown> 1. Ensure that there are no duplicate complaints on your data set </font>

In [11]:
# Ensure that there are no duplicate complaints on your data set
len(master_df["CNCRN_ID"].unique())
# Since unique values are only 35664 and total rows are 35664 - We have no duplicates

35664

### <font color = brown> 2. Check if all the Complaints in excel have a match – those that doesn’t have a match please find the reason why are they dropped.

In [12]:
# Do a outer join on two dataframes using all rows from excel file
temp = pd.merge(teradata_df2, exl_data_df1, on='CNCRN_ID', how='right', indicator=True)
temp.head()

CNCRN_ID     RECVD_D                                           CNCRN_DS  \
0  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...   
1  COM-909747  2019-07-29  In July 2017,ANZ Asset Finance approved a Car ...   
2  COM-909499  2019-07-26  Hasn't received 120,000 frequent flyer points ...   
3  COM-931674  2019-11-12  team,  Can you please review this account for ...   
4  COM-926607  2019-10-15  Registration  Complaint Description There were...   

                                        BCOP_Breach1 BCOP_Breach2  \
0                09 Communication between us and you          NaN   
1               17 A responsible approach to lending          NaN   
2                  08 Providing you with information          NaN   
3  41 We will try to help you if you are experien...          NaN   
4                     04 Trained and competent staff          NaN   

  BCOP_Breach3 BCOP_Breach4  Breaches _merge  
0          NaN          NaN         1   both  
1          NaN          NaN         1   both  
2          NaN          NaN         1   both  
3          NaN          NaN         1   both  
4          NaN          NaN         1   both

In [13]:
temp["_merge"].value_counts()
# There is no missing excel concern id in Teradata

both          302
right_only      0
left_only       0
Name: _merge, dtype: int64

### <font color = brown> 3. Get the ratio of BCOP complaints vs NON-BCOP

In [14]:
# We did outer join on Excel data and teradata data and removed duplicates - This has total count of 35664
Total_CON_ID = master_df.shape[0]
print("Total_CON_ID : ", Total_CON_ID)

Total_CON_ID :  35664


In [15]:
# BCOP = unique rows in Excel file
BCOP = len(exl_data_df1["CNCRN_ID"].unique())
print("BCOP : ", BCOP)

BCOP :  302


In [16]:
# Total number of NON-BCOP concern -id 
NON_BCOP = Total_CON_ID - BCOP
print("NON_BCOP : ", NON_BCOP)

NON_BCOP :  35362


In [17]:
# Ratio of BCOP complaints vs NON-BCOP
Ratio = BCOP/ NON_BCOP
print("Ratio :", Ratio)

Ratio : 0.008540240936598609


### <font color = brown> 4. Create a Target variable by using the  Code chapter breached 1 variable and rename it as breach_type

In [18]:
master_df.head()

CNCRN_ID     RECVD_D                                           CNCRN_DS  \
0  COM-908270  2019-07-19  Case Management- Lending customer requests for...   
1  COM-937313  2019-12-11  You are dissatisfied with delay in assessment ...   
2  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...   
3  COM-936072  2019-12-05  Customer, would like to cancel his Bpay disput...   
4  COM-922281  2019-09-26  I believe in protecting and improving our natu...   

                          BCOP_Breach1 BCOP_Breach2 BCOP_Breach3 BCOP_Breach4  \
0                                  NaN          NaN          NaN          NaN   
1                                  NaN          NaN          NaN          NaN   
2  09 Communication between us and you          NaN          NaN          NaN   
3                                  NaN          NaN          NaN          NaN   
4                                  NaN          NaN          NaN          NaN   

   Breaches     _merge  
0       NaN  left_only  
1       NaN  left_only  
2       1.0       both  
3       NaN  left_only  
4       NaN  left_only

In [19]:
# Populate the Breach type field with BCOP
master_df.loc[master_df['_merge'] == "both", "Breach_Type"] = "BCOP"
# Populate the Breach type field with NBCOP
master_df.loc[master_df['_merge'] != "both", "Breach_Type"] = "NBCOP"

In [20]:
master_df.head()

CNCRN_ID     RECVD_D                                           CNCRN_DS  \
0  COM-908270  2019-07-19  Case Management- Lending customer requests for...   
1  COM-937313  2019-12-11  You are dissatisfied with delay in assessment ...   
2  COM-916025  2019-08-26  Customer had called us to confirm cash withdra...   
3  COM-936072  2019-12-05  Customer, would like to cancel his Bpay disput...   
4  COM-922281  2019-09-26  I believe in protecting and improving our natu...   

                          BCOP_Breach1 BCOP_Breach2 BCOP_Breach3 BCOP_Breach4  \
0                                  NaN          NaN          NaN          NaN   
1                                  NaN          NaN          NaN          NaN   
2  09 Communication between us and you          NaN          NaN          NaN   
3                                  NaN          NaN          NaN          NaN   
4                                  NaN          NaN          NaN          NaN   

   Breaches     _merge Breach_Type  
0       NaN  left_only       NBCOP  
1       NaN  left_only       NBCOP  
2       1.0       both        BCOP  
3       NaN  left_only       NBCOP  
4       NaN  left_only       NBCOP

In [21]:
master_df["Breach_Type"].value_counts()

NBCOP    35362
BCOP       302
Name: Breach_Type, dtype: int64

### <font color = brown> Preprocess the Concern Description field

In [22]:
model_df = master_df[["CNCRN_DS", "Breach_Type"]]
model_df = model_df.iloc[:500]
model_df.head()

CNCRN_DS Breach_Type
0  Case Management- Lending customer requests for...       NBCOP
1  You are dissatisfied with delay in assessment ...       NBCOP
2  Customer had called us to confirm cash withdra...        BCOP
3  Customer, would like to cancel his Bpay disput...       NBCOP
4  I believe in protecting and improving our natu...       NBCOP

In [68]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

#def preprocess(document, stem=True):
def preprocess(document):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [wordnet_lemmatizer.lemmatize(word) for word in words if word not in stopwords.words("english") if word[0].isalpha()== True]
    
    # join words to make sentence
    document = " ".join(words)

    return document

In [69]:
X = list(master_df["CNCRN_DS"].apply(lambda x: preprocess(x)))
y = list(master_df["Breach_Type"])

In [70]:
print(X[:5])

['case management- lending customer request default listing removed claim listing debt sale occurred requesting financial hardship assistance', 'dissatisfied delay assessment anz home loan application reference number also happy unable get update regard assessment application lender aitkenvale branch noted asked provide supporting document assist assessment process facing difficulty providing document mainly due recent public holiday', 'customer called u confirm cash withdrawal could occur anz bourke st mall however customer attended understood withdrawal occur particular branch', 'customer would like cancel bpay dispute received money back merchant', "believe protecting improving natural environment customer expect bank custody money expansion fossil fuel industry australia threatening land water resource people\x1as health part natural environment including iconic site great barrier reef furthermore world point need rapidly transition away polluting fossil fuel clean renewable energy

### <font color = brown> Preparing to create a train and test set

In [71]:
# Split your data X and y, into a training and a test set and fit the pipeline onto the training data
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.3, random_state=101)
print(len(X_train))
print(len(y_train))

24964
24964


In [61]:
#Applying Smote
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_train_bal,y_train_bal = sm.fit_sample(X_train,y_train)
print(X_train_bal.shape)
print(y_train_bal.shape)

ValueError: Expected 2D array, got 1D array instead:
array=['custom two properti secur anz home loan . seek top loan $ 150k consolid debt financi institut . last 12 month support daughter two children invest properti vacant 6 month . inform anz increas meet loan valu ratio . state increas approv need sell properti go bankrupt . resolut seek lend decis overturn .'
 'custom unhappi anz berri branch unabl assist transfer fund deceas wife\x1a account , advis branch advis , fund transfer invest account , need fund transfer urgent , advis custom unabl id , offer connect berri branch , seek anz transfer $ 60,000.00 dda 251310265 dda 5526-37034 * * * * provid inform custom id custom * * * * *'
 'situat receiv call andrew lan ide dean ide * * * anz platinum end * * 7745 master card low rate anz platinum custom receiv letter mail say interest rate increas 11.49 % 20.24'
 ...
 "recent told friend friend anz bank refund custom monthli fee made deposit $ 2000 . recent gone local branch hallett cove

### <font color = brown> Create Bag of words

In [72]:
vectorizer = CountVectorizer()
bow_model = vectorizer.fit_transform(X_train)
print(vectorizer.get_feature_names())

['00', '000', '00009ipkkxq', '000736577', '00219', '00266', '00276', '00328', '0038', '00466', '00503', '00558', '01', '011', '0116', '01318', '0219', '0236', '0412180813', '0446', '048', '05', '06', '07', '08', '09', '0e10378fe110', '10', '100', '10025', '102', '10272', '103', '103931', '105545', '1063', '1067', '11', '111', '1117', '113', '11460102', '117', '12', '120', '121', '1210', '125705', '12ee0d', '12minutes', '130', '132', '139', '140', '142', '144', '14g60', '15', '15028013', '15112418', '1528', '155', '157', '1578253', '1581995620', '1586', '15e148', '16', '1697', '16g77', '17', '1710467', '1711098', '17134', '1727703', '17763', '1783122', '1783329', '1790960', '18', '1802310', '180610', '182', '1821862', '1835503', '18362', '1847253', '1855217', '1858489', '1860631', '1865069', '1868855', '1872211', '1876623', '1888711', '1891156', '1897236', '1897810', '19', '1910696', '1916008', '1917585', '1917862', '1922477', '1923402', '1926021', '1926822', '1928817', '1945114', '1947

### <font color = brown> Preparing to create feature maps for train and test data

In [ ]:
## creating a LazyMap of feature
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
## - creating the feature map of train and test data

training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features, test_messages)

In [ ]:
print(training_set[:5])

In [ ]:
print('Training set size : ', len(training_set))
print('Test set size : ', len(testing_set))

In [ ]:
## Training the classifier with NaiveBayes algorithm
myClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [ ]:
## - Analyzing the accuracy of the test set
print(nltk.classify.accuracy(myClassifier, training_set))

In [ ]:
## Analyzing the accuracy of the test set
print(nltk.classify.accuracy(myClassifier, testing_set))